# Introduction générale

Ce cours a pour but de brosser un tableau très large des différentes méthodes numériques employées usuellement en mécanique. Le profil des étudiants suivant le cours étant particulièrement large, le but est que chacun y trouve son compte. Ainsi, ce cours permettra aux débutants de se forger un culture numérique de base, indispensable à l'ingénieur, et les plus confirmés pourront parfaire leurs connaissances et approfondir certaines notions. Pour guider le lecteur, les titres des sections seront annotées d'étoiles qui indiqueront l'"importance" de la notion abordée dans le bagage de culture générale de l'ingénieur mécanique : 
- _* : notion de base, à maîtriser, constituant un socle de connaissance essentiel pour l'ingénieur,
- _** : notion fréquemment rencontrée dont vous devez avoir entendu parler au moins une fois et pour laquelle vous devez a minima comprendre les grandes lignes, 
- _*** : notion plus avancée, principalement pour les profils étant déjà à l'aise avec les notions abordées.

Dans cette logique de proposer un support relativement large, l'accent sera mis davantage sur l'illustration des différentes méthodes en insistant sur les idées directrices sous-jacentes, que sur la rigueur mathématique des raisonnements proposés. Les méthodes abordées seront généralement suivie d'une implémentation Python en guise d'illustration. On notera en gras les termes de jargon qui font parti du vocabulaire du numéricien.

## Contexte général

En premier lieu, les problèmes de mécanique sont généralement posés sous la forme d'un problème d'analyse constituée d'une équation d'équilibre, munie d'une loi de comportement, faisant le lien entre les contraintes $\mathbf \sigma$ et les déformations $\mathbf \varepsilon$ : 
$$
div \, \mathbf \sigma 
+ \mathbf f
= 0
$$
$$
\mathbf \sigma =
\mathcal A [
\mathbf \varepsilon(t < \tau), \tau]
$$
Le problème consiste alors à trouver un champ, généralement de déplacement $\mathbf u$, qui satisfait en tout point du domaine d'étude l'équation d'équilibre et qui respecte les conditions au limites. Ce problème analytique ne peut pas être traité numériquement sous cette forme. C'est la raison pour laquelle, on utilise des méthodes, généralement  la méthodes des éléments finis (qui ne sera pas traîtée dans ce cours) pour transformer ce problème initial en un problème (souvent d'algèbre) traitable numériquement. Plus généralement, on appelle **analyse numérique** le champ d'étude entre les mathématiques, l'informatique (et souvent la physique) qui aborde les méthodes permettant de résoudre numériquement des problèmes d'analyse.

Dans le cas d'un comportement linéaire, le problème élément finis à résoudre peut se mettre sous la forme usuelle : 
$$
\underbrace{\mathbf K
\cdot
\mathbf U}_{\mathbf F_{int}}
= 
\mathbf F_{ext}
$$
où $\mathbf K \in\mathbb R^{n_{dof}\times n_{dof}}$ est la **matrice de rigidité**, $\mathbf U \in \mathbb R^{n_{dof}}$ le vecteur des $n_{dof}$ inconnues généralement appellées **degrés de libertés du système (dof)** et $\mathbf F_{ext} \in \mathbb R^{n_{dof}}$ (resp. $\mathbf F_{int}$) le vecteur des efforts externes (resp. internes). En adoptant les notations mathématiques usuelles, on cherche $\mathbf x$ tel que : 
$$
\mathbf A \cdot \mathbf x = \mathbf b.
$$
Néanmoins, lorsque le comportement considéré n'est pas linéaire, le problème ne peut pas se mettre sous la forme précédente. On considérera alors le résidu $\mathbf R$ :
$$
\mathbf R(\mathbf U)
= 
\mathbf F_{int}(\mathbf U)
- \mathbf F_{ext}
=
\mathbf 0
$$
et on cherchera le vecteur $\mathbf U$ qu'il l'annule. Les **non-linéarités** complexifient grandement la résolution du problème à tel point qu'il n'est généralement pas possible de calculer $\mathbf U$ directement. On préferera alors décomposer le calcul initial en petits **incréments**, plus simples à résoudre numériquement. Pour ce faire, on décomposera le chargement initial en petits incréments de chargement. Ainsi, on cherchera successivement les incréments de déplacement $\Delta \mathbf U^{1}, \Delta\mathbf U^{2},...$ qui annuleront les résidus $\mathbf R(\Delta\mathbf U^i)$.

On détaille maintenant les différentes méthodes numériques qui permettent la résolution de problèmes de mécanique ainsi posés en insistant sur les pratiques généralement employées par l'ingénieur. Nous verrons ainsi un certain nombre d'algorithme de résolution. Avant de les détailler et de taper la moindre ligne de code, il semble nécessaire d'aborder la notion de compléxité algorithmique.


## Complexité algorithmique *

La notion de **complexité algorithmique** permet d'évaluer la performance d'un algorithme. Il s'agit d'estimer le nombre d'opérations qu'induit un algorithme donné. On exprime généralement la complexité d'un algorithme en fonction de la taille des données d'entrée (nombre de degrés de liberté, taille du tableau à trier,...). On utilise classiquement le notation d'analyse asymptotique $O(\cdot)$, signifiant "dominé par". En pratique, si on dit que l'algorithme est de complexité $O(N^2)$ où $N$ correspond à la taille de la donnée d'entrée, il faut comprendre que le nombre d'opération nécessaire à la réalisation de l'algorithme est de l'ordre de $N^2$. On distingue une complexité en temps (nombre d'opérations nécessaire) et en espace (taille de stockage nécessaire), qu'on ne traitera pas ici.

La réalisation d'une même tâche peut être réalisée via des algorithmes différents, de complexités potentiellement différentes. C'est le cas des algoritmes de tri permettant de ranger un ensemble de nombre du plus petit au plus grand par exemple. Il existe un grand nombre d'algorithmes de tri mais nous allons nous concentrer sur les trois suivants de complexité différentes :
- Tri stupide : tant que les nombres ne sont pas triés, on les mélange aléatoirement (complexité moyenne $O(n!)$),
- Tri par sélection : méthode la plus naturelle, on parcourt la liste, on trouve le nombre le plus petit, on le met au début et on recommence (compléxité $O(n^2)$),
- Tri fusion, de compléxité $O(n\log(n))$ et qu'on ne détaillera pas ici d'un point de vue technique,

où $n$ nombres sont à trier.

Lorsque $n$ est petit la différence n'est pas forcément flagrante pour celui qui réalise l'algorithme. Néanmoins, lorsque $n$ augmente des différences significatives apparaissent. Pour fixer les idées, en prenant $n = 100$, on obtient :
- Tri stupide : $\approx 100! \approx 10^{158}$ opérations,
- Tri par sélection : $\approx 10\,000$ opérations,
- Tri fusion : $\approx 200$ opérations.

Le nombre d'opérations nécessaires pour réaliser le tri différe donc radicalement d'un algorithme à l'autre. Ce petit exemple montre qu'il est absolument nécessaire de réflechir à la structure de son algorithme sur papier avant d'envisager une implémentation numérique, particulièrement lorsque les données d'entrées sont de taille importante.

Pour débuter, compter le nombre de boucles "for" imbriquées permet souvent d'évaluer la compléxité d'un algorithme.
    En pratique, on évitera d'utiliser des algorithmes de complexité $O(n^3)$ ou supérieure car ils deviennent très lourds lorsque la taille d'entrée augmente. On dit que l'algorithme n'est pas **scalable** (ou **extensible)**. Néanmoins, il n'est pas toujours possible d'améliorer la complexité d'un algorithme. On peut alors se tourner vers des méthodes de **parallélisation** du code pour améliorer les performances.

## Complexité des produits usuels **

Pour illustrer la notion de complexité, on s'intéresse ici à celle de produits usuels en commençant par le produit scalaire (produit vecteur-vecteur). Le produit scalaire de deux vecteur $\mathbf a$ et $\mathbf b$ de taille $n$ s'écrit : 
$$
\mathbf a \cdot \mathbf b
=
\sum_{i=1}^n a_ib_i
$$
L'algorithme correspondant s'écrit :

    s = 0
    Pour i allant de 1 à n faire :
        s = s + ai*bi
    Renvoyer s
Cette algoritme ne nécessite que de parcourir une fois chaque vecteur : il est de complexité $O(n)$.

On s'intéresse au produit matrice-vecteur $\mathbf M\cdot \mathbf v$ avec $\mathbf M \in \mathbb R^{n\times  n}$ et $\mathbf v\in\mathbb R^{n\times  1}$. Le résultat est un vecteur $\mathbf r$ de taille $n$ où la composante $v_i$ est le produit scalaire entre la ligne $i$ de la matrice $\mathbf M$ et le vecteur $\mathbf v$ :
$$
\mathbf v = 
\begin{pmatrix}
lgn_1(\mathbf M)\cdot \mathbf v\\
\vdots\\
lgn_n(\mathbf M)\cdot \mathbf v
\end{pmatrix}
$$
On réalise donc $n$ produits scalaires lors d'un produit matrice-vecteur. La complexité associée est en $O(n\times n) = O(n^2)$. On a donc augmenter l'ordre de complexité du produit associé.

Enfin, en raisonnant de même, le produit matrice-matrice équivaut à $n$ produits matrice-vecteur soit une complexité de $O(n\times n^2) = O(n^3)$. On constate donc que le coût inhérent à un produit matrice-matrice est particulièrement élevé. On comprend ainsi pourquoi on prefère utiliser des méthodes faisant intervenir des produits matrice-vecteur que des produits matrice-matrice, typiquement les méthodes itératives à la place des méthodes directes : nous verrons cela un peu tard dans le cours! 

On peut alors se demander : quelles options a-t-on pour rendre le produit matrice-matrice performant?
- Améliorer l'algorithme, par exemple en tirant partie de la nature des matrices considérées,
- Utiliser un langage performant,
- Paralléliser les tâches.

C'est la stratégie qu'emploie les bibliothèques classiques, typiquement NumPy, qui optimisent ainsi ce type d'opérations. Nous allons maintenant refaire un rappel de Python, langage qu'on choisira pour traduire nos algorithmes, en insistant sur l'emploi des bibliothèques de calcul scientifique.